##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [8]:
# -*- coding: utf8 -*-
import codecs
import re
import csv
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
debug=0

#Préparation du lexique

###Lecture des lexiques

In [9]:
nomLexique="/Users/gilles/Copy/Python/phonemisation/bdlexique-PDM.txt"
with codecs.open(nomLexique, 'r',encoding='utf8') as infile:
    bdlexique=infile.readlines()
for ligne in bdlexique[:5]:
    (ortho,phono,ext,cs,ms,vs,lexeme,L23,lemmeFrantext,lemmeFilms,formeFrantext,formeFilms)=ligne.strip().split(";")
    print ortho,cs,ms,vs,lexeme,lemmeFrantext,lemmeFilms,formeFrantext,formeFilms

a N Mj  a 8136 5865 8136 5865
à p   à 1219040 1920905 1219040 1920905
abaca N MS  abaca 1 0 1 0
abacas N MP  abaca *** *** *** ***
abacule N MS  abacule *** *** *** ***


In [10]:
nomLexiqueOrg="/Users/gilles/Copy/Python/phonemisation/Lexique380-UTF8.txt"
verbes={}
vlemmes={}
noms={}
nlemmes={}
adjectifs={}
alemmes={}
with codecs.open(nomLexiqueOrg, 'r',encoding='utf8') as infile:
    lexiqueorg=infile.readlines()
for ligne in lexiqueorg[1:]:
    morceaux=ligne.strip().split("\t")
    (ortho,phon,lemme,cgram,genre,nombre,freqlemfilms2,freqlemlivres,freqfilms2,freqlivres,infover)=morceaux[:11]
    freqlemfilms2=str(int(100*float(freqlemfilms2)))
    freqlemlivres=str(int(100*float(freqlemlivres)))
    freqfilms2=str(int(100*float(freqfilms2)))
    freqlivres=str(int(100*float(freqlivres)))
    if cgram=="NOM":
        vs=genre.upper()+nombre.upper()
        nom=";".join([ortho,lemme,vs])
        noms[nom]=(freqlemfilms2,freqlemlivres,freqfilms2,freqlivres)
        if not lemme in nlemmes:
            nlemmes[lemme]=(freqlemfilms2,freqlemlivres)
#        print nom, noms[nom]
    elif cgram=="ADJ":
        vs=genre.upper()+nombre.upper()
        adjectif=";".join([ortho,lemme,vs])
        adjectifs[adjectif]=(freqlemfilms2,freqlemlivres,freqfilms2,freqlivres)
        if not lemme in alemmes:
            alemmes[lemme]=(freqlemfilms2,freqlemlivres)
#        print adjectif, adjectifs[adjectif]
    elif cgram=="VER":
#        print ortho,lemme,genre,nombre,infover
        if infover=="ind:pre:2p;inf;":
            infover="inf;"
        tams=infover.rstrip(";").split(";")
        for tam in tams:
            morceaux=tam.split(":")
#            print morceaux
            if len(morceaux)==3:
                if morceaux[0]=="imp":
                    mode="I"
                else:
                    mode=morceaux[0][0]
                if morceaux[1]=="pas":
                    temps="a"
                else:
                    temps=morceaux[1][0]
                infotam=temps+mode+morceaux[2].upper()
            elif len(morceaux)==2:
                if morceaux[1]=="pre":
                    infotam=morceaux[0][0]+"P"
                elif morceaux[1]=="pas":
                    infotam="pp"+genre.upper()+nombre.upper()
            elif len(morceaux)==1:
                infotam=morceaux[0]
            verbe=";".join([ortho,lemme,infotam])
 #           print verbe
            verbes[verbe]=(freqlemfilms2,freqlemlivres,freqfilms2,freqlivres)
            if not lemme in vlemmes:
                vlemmes[lemme]=(freqlemfilms2,freqlemlivres)
#            print verbe, verbes[verbe]

In [11]:
resync=[]
for ligne in bdlexique[:]:
    nouvelleLigne=ligne.strip()
    try :
        (ortho,phono,ext,cs,ms,vs,lexeme,L23,lemmeFrantext,lemmeFilms,formeFrantext,formeFilms)=ligne.strip().split(";")
    except ValueError:
        print ligne
#    print ortho, lexeme, lemmeFrantext,lemmeFilms,formeFrantext,formeFilms
    if lexeme[-1] in "1234":
        lexeme=lexeme[:-1]
    if lexeme in [u"dépendre"]: 
        debug=1
    else:
        debug=0
    if cs in ["V","K"]:
        verbe=";".join([ortho,lexeme,vs+ms])
        if verbe in verbes:
            if lemmeFrantext!=verbes[verbe][0] and lemmeFilms!=verbes[verbe][1] and formeFrantext!=verbes[verbe][2] and formeFilms!=verbes[verbe][3]:
                if debug: print "different"
                if debug: print verbe, verbes[verbe]
            nouvelleLigne=";".join([ortho,phono,ext,cs,ms,vs,lexeme,L23,verbes[verbe][0],verbes[verbe][1],verbes[verbe][2],verbes[verbe][3]])
        elif lexeme in vlemmes:
            if debug: print vlemmes[lexeme]
            nouvelleLigne=";".join([ortho,phono,ext,cs,ms,vs,lexeme,L23,vlemmes[lexeme][0],vlemmes[lexeme][1],"***","***"])
        else:
            nouvelleLigne=";".join([ortho,phono,ext,cs,ms,vs,lexeme,L23,"***","***","***","***"])
    elif cs in ["G","H","I","J"]:
        adjectif=";".join([ortho,lexeme,ms])
        if adjectif in adjectifs:
#            print adjectifs[adjectif]
            nouvelleLigne=";".join([ortho,phono,ext,cs,ms,vs,lexeme,L23,adjectifs[adjectif][0],adjectifs[adjectif][1],adjectifs[adjectif][2],adjectifs[adjectif][3]])
        elif lexeme in alemmes:
#            print alemmes[lexeme]
            nouvelleLigne=";".join([ortho,phono,ext,cs,ms,vs,lexeme,L23,alemmes[lexeme][0],alemmes[lexeme][1],formeFrantext,formeFilms])
    elif cs in ["N"]:
        nom=";".join([ortho,lexeme,ms])
        if nom in noms:
#            print noms[nom]
            nouvelleLigne=";".join([ortho,phono,ext,cs,ms,vs,lexeme,L23,noms[nom][0],noms[nom][1],noms[nom][2],noms[nom][3]])
        elif lexeme in nlemmes:
#            print nlemmes[lexeme]
            nouvelleLigne=";".join([ortho,phono,ext,cs,ms,vs,lexeme,L23,nlemmes[lexeme][0],nlemmes[lexeme][1],formeFrantext,formeFilms])
    if debug: print [ortho, nouvelleLigne]
    resync.append(nouvelleLigne)
    

[u'd\xe9pendre', u'd\xe9pendre;d\xe8p\xe2dr;@;V;;inf;d\xe9pendre;L23;6033;3649;246;284']
[u'd\xe9pends', u'd\xe9pends;d\xe8p\xe2;z";V;1S;pi;d\xe9pendre;L23;6033;3649;118;20']
[u'd\xe9pends', u'd\xe9pends;d\xe8p\xe2;z";V;2S;pi;d\xe9pendre;L23;6033;3649;118;20']
[u'd\xe9pend', u'd\xe9pend;d\xe8p\xe2;t";V;3S;pi;d\xe9pendre;L23;6033;3649;4830;1520']
[u'd\xe9pendons', u'd\xe9pendons;d\xe8p\xe2d\xf4;z";V;1P;pi;d\xe9pendre;L23;6033;3649;43;20']
[u'd\xe9pendez', u'd\xe9pendez;d\xe8p\xe2de;z";V;2P;pi;d\xe9pendre;L23;6033;3649;25;0']
[u'd\xe9pendent', u'd\xe9pendent;d\xe8p\xe2d;@t";V;3P;pi;d\xe9pendre;L23;6033;3649;264;209']
[u'd\xe9pendais', u'd\xe9pendais;d\xe8p\xe2d\xe8;z";V;1S;ii;d\xe9pendre;L23;6033;3649;4;34']
[u'd\xe9pendais', u'd\xe9pendais;d\xe8p\xe2d\xe8;z";V;2S;ii;d\xe9pendre;L23;6033;3649;4;34']
[u'd\xe9pendait', u'd\xe9pendait;d\xe8p\xe2d\xe8;t";V;3S;ii;d\xe9pendre;L23;6033;3649;187;764']
[u'd\xe9pendions', u'd\xe9pendions;d\xe8p\xe2dj\xf4;z";V;1P;ii;d\xe9pendre;L23;6033;3649;1;7']


In [12]:
nomLexiqueResync=nomLexique.split(".")[0]+"-Resync.txt"
resync=sorted([x.rstrip("\n")+"\n" for x in set(resync)])
with codecs.open(nomLexiqueResync, 'w',encoding='utf8') as outfile:
    outfile.writelines(resync)


In [13]:
resync[11800:12000]

[u'aiguillonneriez;\xe8gHij\xf2n6rje;z";V;2P;pc;aiguillonner;L23;17;155;***;***\n',
 u'aiguillonnerions;\xe8gHij\xf2n6rj\xf4;z";V;1P;pc;aiguillonner;L23;17;155;***;***\n',
 u'aiguillonnerons;\xe8gHij\xf2n6r\xf4;z";V;1P;fi;aiguillonner;L23;17;155;***;***\n',
 u'aiguillonneront;\xe8gHij\xf2n6r\xf4;t";V;3P;fi;aiguillonner;L23;17;155;***;***\n',
 u'aiguillonnes;\xe8gHij\xf2n;@;V;2S;pi;aiguillonner;L23;17;155;***;***\n',
 u'aiguillonnes;\xe8gHij\xf2n;@;V;2S;ps;aiguillonner;L23;17;155;***;***\n',
 u'aiguillonnez;\xe8gHij\xf2ne;z";V;2P;pI;aiguillonner;L23;17;155;***;***\n',
 u'aiguillonnez;\xe8gHij\xf2ne;z";V;2P;pi;aiguillonner;L23;17;155;***;***\n',
 u'aiguillonniez;\xe8gHij\xf2nje;z";V;2P;ii;aiguillonner;L23;17;155;***;***\n',
 u'aiguillonniez;\xe8gHij\xf2nje;z";V;2P;ps;aiguillonner;L23;17;155;***;***\n',
 u'aiguillonnions;\xe8gHij\xf2nj\xf4;z";V;1P;ii;aiguillonner;L23;17;155;***;***\n',
 u'aiguillonnions;\xe8gHij\xf2nj\xf4;z";V;1P;ps;aiguillonner;L23;17;155;***;***\n',
 u'aiguillonnons;\xe